<a href="https://colab.research.google.com/github/hammaduu/LLMs/blob/main/InvoiceBot_Pilot_Cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain-groq langchain langchain_core "langchain-chroma>=0.1.2" langchain_community sentence_transformers fastembed faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.4/438.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 94.4 MB/s eta 0:00:0

In [2]:
# Get API Key
import getpass, os
os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [3]:
# Import LLM
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama3-70b-8192", temperature=0, max_tokens=1024)

In [67]:
import pandas as pd
import random
import numpy as np
from datetime import datetime
from faker import Faker

fake = Faker()

# Generate 10 unique mock B2B customers
customers = [fake.company() for _ in range(10)]
months = ["2024-03", "2024-04"]

# Invoice summary (aggregated by customer per month)
invoice_summary_data = []
for customer in customers:
    for month in months:
        invoice_summary_data.append({
            "customer_name": customer,
            "invoice_month": month,
            "total_amount": random.randint(80000, 180000),
            "msisdn_count": random.randint(25, 75)
        })

invoice_summary_df = pd.DataFrame(invoice_summary_data)

# Generate ~50 MSISDNs with customer association
msisdns = []
for customer in customers:
    count = random.randint(5, 10)
    for _ in range(count):
        msisdns.append({
            "customer_name": customer,
            "msisdn": "9230" + ''.join([str(random.randint(0, 9)) for _ in range(7)])
        })

# Generate per-day MSISDN usage for March and April 2024
usage_data = []
months = ["2024-03", "2024-04"]
for record in msisdns:
    for month in months:
        for _ in range(random.randint(3, 7)):  # simulate multiple records per MSISDN
            day = random.randint(1, 28)
            usage_data.append({
                "msisdn": record["msisdn"],
                "customer_name": record["customer_name"],
                "date": f"{month}-{day:02}",
                "data_mb": random.randint(100, 5000),
                "calls": random.randint(0, 50),
                "sms": random.randint(0, 20),
                "location": fake.city()
            })

msisdn_usage_df = pd.DataFrame(usage_data)

# Preview samples
invoice_summary_df.head(), msisdn_usage_df.head()

(             customer_name invoice_month  total_amount  msisdn_count
 0              Chapman Ltd       2024-03        141999            58
 1              Chapman Ltd       2024-04        103042            72
 2  Hancock, Horne and Rush       2024-03        172231            52
 3  Hancock, Horne and Rush       2024-04         98788            49
 4   Walsh, King and Wright       2024-03        149037            59,
         msisdn customer_name        date  data_mb  calls  sms  \
 0  92306556418   Chapman Ltd  2024-03-21     2379     19    8   
 1  92306556418   Chapman Ltd  2024-03-04     1567     45   15   
 2  92306556418   Chapman Ltd  2024-03-02     4983     50   17   
 3  92306556418   Chapman Ltd  2024-03-26      569     32   19   
 4  92306556418   Chapman Ltd  2024-03-21     2890     29    5   
 
               location  
 0         Youngborough  
 1     East Jeffreyfort  
 2             Hoodbury  
 3  East Williamchester  
 4          Johnsonberg  )

In [58]:
# Tool Functions
def get_invoice_change(customer_name: str, month1: str, month2: str):
    customer_name = customer_name.lower().strip()
    month1 = month1.strip().strip("'\"")
    month2 = month2.strip().strip("'\"")

    # Invoice summary
    df = invoice_summary_df.copy()
    df["invoice_month"] = df["invoice_month"].astype(str).str.strip().str[:7]
    df["customer_name"] = df["customer_name"].astype(str).str.lower().str.strip()
    df_filtered = df[(df["customer_name"] == customer_name) & (df["invoice_month"].isin([month1, month2]))]

    if len(df_filtered) != 2:
        return f"Could not find data for both months: {month1}, {month2}"

    values = df_filtered.set_index("invoice_month")["total_amount"].to_dict()
    msisdn_counts = df_filtered.set_index("invoice_month")["msisdn_count"].to_dict()
    diff = values[month2] - values[month1]
    pct = round((diff / values[month1]) * 100, 2)

    base = f"{customer_name.title()}'s invoice changed from {values[month1]} in {month1} to {values[month2]} in {month2} — a change of {diff} ({pct}%)."

    # Usage data
    usage_df = msisdn_usage_df.copy()
    usage_df["customer_name"] = usage_df["customer_name"].str.lower().str.strip()
    usage_df["month"] = usage_df["date"].str[:7]

    usage_month1 = usage_df[(usage_df["customer_name"] == customer_name) & (usage_df["month"] == month1)]
    usage_month2 = usage_df[(usage_df["customer_name"] == customer_name) & (usage_df["month"] == month2)]

    if usage_month1.empty or usage_month2.empty:
        df_month_only = df[df["invoice_month"].isin([month1, month2])]
        print(df_month_only)
        return base + " However, detailed usage data was unavailable to explain the reason."

    # Total usage comparisons
    def summarize(df):
        return {
            "data_mb": df["data_mb"].sum(),
            "calls": df["calls"].sum(),
            "sms": df["sms"].sum(),
            "avg_data_per_user": df.groupby("msisdn")["data_mb"].sum().mean()
        }

    sum1 = summarize(usage_month1)
    sum2 = summarize(usage_month2)

    reasons = []

    if msisdn_counts[month2] > msisdn_counts[month1]:
        reasons.append(f"increased MSISDN count from {msisdn_counts[month1]} to {msisdn_counts[month2]}")
    elif msisdn_counts[month2] < msisdn_counts[month1]:
        reasons.append(f"reduced MSISDN count from {msisdn_counts[month1]} to {msisdn_counts[month2]}")

    if sum2["data_mb"] > sum1["data_mb"]:
        reasons.append(f"increased total data usage ({sum1['data_mb']}MB → {sum2['data_mb']}MB)")
    elif sum2["data_mb"] < sum1["data_mb"]:
        reasons.append(f"reduced total data usage ({sum1['data_mb']}MB → {sum2['data_mb']}MB)")

    if sum2["calls"] > sum1["calls"]:
        reasons.append(f"increased total call volume ({sum1['calls']} → {sum2['calls']})")
    elif sum2["calls"] < sum1["calls"]:
        reasons.append(f"reduced total call volume ({sum1['calls']} → {sum2['calls']})")

    if sum2["sms"] > sum1["sms"]:
        reasons.append(f"increased total SMS ({sum1['sms']} → {sum2['sms']})")
    elif sum2["sms"] < sum1["sms"]:
        reasons.append(f"reduced total SMS ({sum1['sms']} → {sum2['sms']})")

    if sum2["avg_data_per_user"] > sum1["avg_data_per_user"]:
        reasons.append("higher average data usage per MSISDN")
    elif sum2["avg_data_per_user"] < sum1["avg_data_per_user"]:
        reasons.append("lower average data usage per MSISDN")

    reason_text = "Reason: " + "; ".join(reasons).capitalize() + "."
    return base + "\n" + reason_text

def get_top_msisdns_by_data(customer_name: str, month: str, top_n: int = 5):
    customer_name = customer_name.lower().strip()
    df = msisdn_usage_df.copy()
    df["customer_name"] = df["customer_name"].str.lower().str.strip()
    df["month"] = df["date"].str[:7] if df["date"].dtype == "O" else df["date"].dt.strftime("%Y-%m")
    msisdn_df = df[(df["customer_name"] == customer_name) & (df["month"] == month)]
    if msisdn_df.empty:
        return f"No usage data found for '{customer_name.title()}' in {month}."
    totals = msisdn_df.groupby("msisdn")["data_mb"].sum().sort_values(ascending=False).head(top_n)
    return f"Top {top_n} MSISDNs by data usage for {customer_name.title()} in {month}:" + totals.reset_index().to_string(index=False)

In [45]:
# Wrappers
from langchain.tools import Tool

def safe_invoice_change_wrapper(input_str: str):
    try:
        parts = input_str.split(",", 2)
        if len(parts) < 3:
            return "[Tool Error - GetInvoiceChange] Invalid input. Expected format: Customer, YYYY-MM, YYYY-MM"
        customer = parts[0].strip(" '\"\n").strip()
        month1 = parts[1].strip(" '\"\n").strip()
        month2 = parts[2].split()[0].strip(" '\"\n").strip()
        return get_invoice_change(customer, month1, month2)
    except Exception as e:
        return f"[Tool Error - GetInvoiceChange] {str(e)}"

def safe_top_msisdns_wrapper(input_str: str):
    try:
        parts = [x.strip().strip("'").strip('"') for x in input_str.split(",")]
        customer = parts[0]
        month = parts[1]
        return get_top_msisdns_by_data(customer, month)
    except Exception as e:
        return f"[Tool Error - GetTopMSISDNs] {str(e)}"

# Tool registry
tools = [
    Tool(
        name="GetInvoiceChange",
        func=safe_invoice_change_wrapper,
        description="Compare invoice totals between two months. Input: 'Customer Name,YYYY-MM,YYYY-MM'",
        return_direct=True
    ),
    Tool(
        name="GetTopMSISDNs",
        func=safe_top_msisdns_wrapper,
        description="Get top MSISDNs by data usage. Input: 'Customer Name,YYYY-MM'",
        return_direct=True
    )
]

In [46]:
# Prompt template wiring
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, AgentType

prompt = PromptTemplate(
    input_variables=["input"],
    template="""
You are a telecom invoice analyst. Given a customer name and two months, reason about invoice change.
Use tool `GetInvoiceChange` to compare invoices, then `GetTopMSISDNs` to explore usage.

Question: {input}
"""
)

agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_kwargs={"prefix": prompt.template},
    verbose=False
)

In [68]:
# Sample runs
response = agent_executor.run("Why did invoice increase for Chapman Ltd between 2024-03 and 2024-04?")
print(response)

Chapman Ltd's invoice changed from 141999 in 2024-03 to 103042 in 2024-04 — a change of -38957 (-27.43%).
Reason: Increased msisdn count from 58 to 72; reduced total data usage (71389mb → 64040mb); reduced total call volume (631 → 570); reduced total sms (287 → 206); lower average data usage per msisdn.


In [29]:
response = agent_executor.run("Why did invoice increase for Scott-Ochoa between 2024-03 and 2024-04?")
print(response)

1. Larsen, Jenkins And Martinez
2. Pittman, Hernandez And Nguyen
3. Scott-Ochoa
4. Martinez, Maynard And Jones
5. Jordan-Simmons
6. Stone-Lee
7. Johnson Inc
8. May Ltd
9. Silva Inc
10. Freeman And Sons
Scott-Ochoa's invoice changed from 96620 in 2024-03 to 108665 in 2024-04 — a change of 12045 (12.47%).


In [61]:
agent_executor.run("List top 5 MSISDNs by data usage for bowen plc in 2024-04.")

'Top 5 MSISDNs by data usage for Bowen Plc in 2024-04:     msisdn  data_mb\n92307444499    17723\n92307134453    15583\n92309598876    10779\n92306361143     8232\n92303836383     4129'

In [65]:
msisdn_usage_df[
    #(msisdn_usage_df["customer_name"].str.lower().str.strip() == "bowen plc") &
    (msisdn_usage_df["date"].str.startswith("2024-04"))
]

,msisdn,customer_name,date,data_mb,calls,sms,location
0,92306349355,"Allen, Adams and Taylor",2024-04-14,3215,30,6,Feliciaville
1,92306349355,"Allen, Adams and Taylor",2024-04-14,2629,1,5,Barrberg
2,92306349355,"Allen, Adams and Taylor",2024-04-03,108,28,18,Port Alison
3,92306349355,"Allen, Adams and Taylor",2024-04-25,3798,45,14,South Angela
4,92304616336,"Allen, Adams and Taylor",2024-04-06,4735,12,12,New Mark
...,...,...,...,...,...,...,...
354,92307079372,Cox-Hawkins,2024-04-12,1242,29,12,Lake Edwardmouth
355,92307079372,Cox-Hawkins,2024-04-05,1431,6,10,Smithbury
356,92307079372,Cox-Hawkins,2024-04-28,917,21,18,North Karen
357,92307079372,Cox-Hawkins,2024-04-04,907,44,19,Florestown
